In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Load the original image and the saliency map
original_image_path = '/content/27.jpg'  # Update with your original image path
saliency_map_path = '/content/27_tempsal.jpg'  # Update with your saliency map path

original_image = Image.open(original_image_path)
saliency_map = Image.open(saliency_map_path).convert('L')  # Convert saliency map to grayscale

# Resize the saliency map to match the original image size if different
if original_image.size != saliency_map.size:
    saliency_map = saliency_map.resize(original_image.size, Image.Resampling.LANCZOS)

# Convert the image data to an array for plotting
saliency_array = np.array(saliency_map)

# Determine the dpi of the figure to match the original image size
dpi = 100
fig_width, fig_height = original_image.size
fig_size = (fig_width / dpi, fig_height / dpi)  # Size in inches

# Create a figure with the exact size and dpi
fig, ax = plt.subplots(figsize=fig_size, dpi=dpi)
ax.imshow(saliency_array, cmap='hot')
ax.axis('off')  # Hide the axis

# Save the modified image without padding
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.savefig('/content/27_tempsal.jpg', dpi=dpi)
plt.close(fig)

In [ ]:
import cv2
import numpy as np

def overlay_heatmap_on_image(original_img_path, heatmap_img_path, output_img_path):
    # Read the original image
    orig_image = cv2.imread(original_img_path)
    orig_size = orig_image.shape[:2]  # Height, Width

    # Read the heatmap image (assumed to be already in color)
    overlay_heatmap = cv2.imread(heatmap_img_path)

    # Resize the heatmap to match the original image size
    overlay_heatmap = cv2.resize(overlay_heatmap, (orig_size[1], orig_size[0]))

    # Overlay the heatmap on the original image
    overlay_heatmap = cv2.applyColorMap(overlay_heatmap, cv2.COLORMAP_JET)
    overlay_image = cv2.addWeighted(orig_image, 1, overlay_heatmap, 0.8, 0)

    # Save the result
    cv2.imwrite(output_img_path, overlay_image)

# Example usage:
original_img_path = '/content/27.jpg'
heatmap_img_path = '/content/27_tempsal.jpg'
output_img_path = heatmap_img_path

overlay_heatmap_on_image(original_img_path, heatmap_img_path, output_img_path)


In [1]:
!git clone https://github.com/samyak0210/saliency
%cd saliency

Cloning into 'saliency'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 156 (delta 47), reused 36 (delta 22), pack-reused 84
Receiving objects: 100% (156/156), 1.32 MiB | 5.15 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/saliency


In [ ]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement apturl==0.5.2 (from versions: none)
ERROR: No matching distribution found for apturl==0.5.2


In [2]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 11.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sh

In [3]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

params = {
    'save_path': '/content/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
}

ses = lt.session()
ses.listen_on(6881, 6891)
link = input("Input Torrent file link or Magnet link and Press Enter: ")

if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    if os.path.exists('torrent.torrent'):
        os.remove('torrent.torrent')

    wget.download(link, 'torrent.torrent')
    t = Torrent.read('torrent.torrent')
    link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))


print(link)
handle = lt.add_magnet_uri(ses, link, params)
# change the 0 to a 1 to download sequentially
handle.set_sequential_download(0)
ses.start_dht()
begin = time.time()

print(datetime.datetime.now())
print('Downloading Metadata...')

while not handle.has_metadata():
    time.sleep(1)

print('Got Metadata, Starting Torrent Download...')
print("Starting", handle.name())

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())

magnet:?xt=urn:btih:23112f5ae692658388ebfe0db3973e552b95cad9&dn=saved_models.tar&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce
2024-04-24 09:44:44.589144
Got Metadata, Starting Torrent Download...
Starting saved_models.tar
0.00% complete (down: 12.6 kb/s up: 0.6 kB/s peers: 1) downloading 
0.32% complete (down: 668.7 kb/s up: 22.7 kB/s peers: 1) downloading 
1.14% complete (down: 1933.3 kb/s up: 64.0 kB/s peers: 1) downloading 
2.19% complete (down: 2946.6 kb/s up: 98.4 kB/s peers: 1) downloading 
3.24% complete (down: 3379.9 kb/s up: 112.4 kB/s peers: 1) downloading 
3.91% complete (down: 2573.6 kb/s up: 84.2 kB/s peers: 1) downloading 
4.73% complete (down: 2609.5 kb/s up: 85.9 kB/s peers: 1) downloading 
5.69% complete (down: 2938.0 kb/s up: 97.9 kB/s peers: 1) downloading 
6.77% complete (down: 3327.1 kb/s up: 111.5 kB/s peers: 1) downloading 
7.98% complete (down: 3703.2 kb/s up: 123.9 kB/s peers: 1) downloading 
9.34% complete (down: 4130.9 kb/s up: 137.8 kB/s peers: 1)

In [4]:
!mv "/content/Torrent/saved_models.tar" "/content/saliency/SimpleNet/"

In [5]:
%cd SimpleNet

/content/saliency/SimpleNet


In [6]:
!tar -xvf "saved_models.tar" -C "./"

./saved_models/
./saved_models/cat_pnas.pt
./saved_models/mit_pnas.pt
./saved_models/salicon_pnas.pt
./saved_models/salicon_densenet.pt


In [7]:
!pip install -qq pytorch_ssim

  Preparing metadata (setup.py) ... done
DEPRECATION: libtorrent 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of libtorrent or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [17]:
!python test.py --val_img_dir /content/saliency/images --results_dir /content/saliency/results --model_val_path /content/saliency/SimpleNet/saved_models/salicon_densenet.pt

DenseNet Model
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even